## CS2710 Assignment 1 Scratch Work
### Jacob Emmerson
Due: 09.14.23

Python 3.11.4
Conda Environment

In [1]:
from collections import deque
from random import *

NO_MOVE = -1
NULL = -1

In [2]:
# definition of the Puzzle 8 problem
class Puzzle8_Problem:
    initial_state=''
    move_cost=1
    end_state=(1,2,3,4,5,6,7,8,0) #defines end state
    valid_moves=[
         [NO_MOVE,3,NO_MOVE,1],
		 [NO_MOVE,4,0,2],
		 [NO_MOVE,5,1,NO_MOVE],
		 [0,6,NO_MOVE,4],
		 [1,7,3,5],
		 [2,8,4,NO_MOVE],
		 [3,NO_MOVE,NO_MOVE,7],
		 [4,NO_MOVE,6,8],
		 [5,NO_MOVE,7,NO_MOVE]
     ]
    
    def __init__(self, init_state=(1,2,3,4,5,6,7,8,0)):
        self.initial_state=init_state
        
    def shuffle_state(self,k):
        pick=self.end_state
        for i in range(k):
           new=self.generate_all_neighbor_states(pick)
           pick , cost=random.sample(new,1)[0]
           print(pick)
        return pick

    def goal_condition(self,state):
        return (state == self.end_state)
        
    def generate_all_neighbor_states(self, state):
        res=[]
        zero_position=state.index(0)
        for i in range(4):
            next_position=self.valid_moves[zero_position][i]
            if next_position != NO_MOVE:
                new_state=list(state)
                new_state[zero_position], new_state[next_position] = state[next_position], state[zero_position]
                # print(new_state)
                res.append([tuple(new_state), self.move_cost])
        return res
    
    def h_function(self,state):
        return 0

In [3]:
# definition of the TreeNode 
# defines a node of the search tree
class TreeNode:

    problem=NULL
    state=''
    parent = NULL
    f=0
    g=0
    h=0   # values of f, g and h functions related to path

    def __init__(self, problem, state, parent=0,g_value=0):
       self.problem=problem
       self.state=state
       self.parent = parent
       self.g=g_value
       self.h=problem.h_function(state)
       self.f=self.g+self.h
    
    # checks if the node is a goal
    def goalp(self):
        pr=self.problem
        return pr.goal_condition(self.state)
    
    # generates a list of new tree nodes to be added to the search fringe
    def generate_new_tree_nodes(self):   
        new_neighbors = (self.problem).generate_all_neighbor_states(self.state)
        res=[]
        for new_state in new_neighbors:
            new_node=TreeNode(self.problem, new_state[0], self, self.g+new_state[1])
            res.append(new_node)           
        return res
    
    # prints a state associated with a node
    def print_state(self):
        for j in range(0,9,3):
            print((self.state[j],self.state[j+1],self.state[j+2]))
        print(' ')
    
    # returns a path (an ordered list of tree nodes) from the root to this node
    def path(self):
        node, path_back = self, []
        while node:
            path_back.append(node)
            node = node.parent
        return list(reversed(path_back))


In [4]:
# definition of the hash table that can be used to store Puzzle 8 states 
# use it to check for state repeats
class HashTable:
      hash={}
      
      def __init__(self):
          self.hash={}
      
      def in_hashp(self, item):
          return (item in self.hash)
      
      def add_hash(self, item, value=0):
          self.hash[item]=value
          
      def get_hash_value(self, item):
          if item in self.hash:
              return self.hash[item]
          else:
              return -1
      def delete_hash(self):
          del(self.hash)
          self.hash={}
      
def print_path(path):
    for node in path:
        node.print_state()


Example1=(1,2,3,4,6,0,7,5,8)
Example2=(2,3,0,1,5,6,4,7,8)
Example3=(4,1,2,7,6,3,0,5,8)
Example4=(4,1,2,7,6,3,5,8,0)
Example5=(5,3,0,4,7,6,2,1,8)

In [5]:
def breadth_first_search(problem):
     queue =deque([])
     root=TreeNode(problem,problem.initial_state)
     queue.append(root) # push initial state in queue
     while len(queue)>0: 
         next=queue.popleft()
         if next.goalp(): # does the current state satisfy the goal condition? (defined by TreeNode)
             del(queue) 
             return next.path() # solution
         else:
             new_nodes=next.generate_new_tree_nodes() 
             for new_node in new_nodes:
                  queue.append(new_node)         
     print('No solution')
     return NULL


In [6]:
def breadth_first_search_stats(problem):
    n_expanded = 0
    n_generated = 0
    queue_len = 0
    path_len = 0
    queue =deque([])
    root=TreeNode(problem,problem.initial_state)
    queue.append(root) # push initial state in queue
    while len(queue)>0:
        if len(queue) > queue_len:
            queue_len = len(queue) # finds max queue length

        next=queue.popleft()
        n_expanded += 1 # popping indicates expansion

        if next.goalp(): # does the current state satisfy the goal condition? (defined by TreeNode)
            del(queue) 
            path_len = len(next.path())
            return {
                'path' : next.path(),
                'Nodes Expanded' : n_expanded,
                'Nodes Generated' : n_generated,
                'Max Queue Length' : queue_len,
                'Path Length' : path_len
            }
        else:
            new_nodes=next.generate_new_tree_nodes() 
            n_generated += len(new_nodes) # generate_new_tree_nodes -> lst[], 
            for new_node in new_nodes:
                queue.append(new_node)         
    print('No solution')
    return NULL

In [52]:
def check_cyclic_repeats(node):
    stop = len(node.path()) - 1
    for n in node.path()[:stop]:
        if n.state == node.state:
            return True
        
    return False

In [53]:
def breadth_first_search_cycles(problem):
    n_expanded = 0
    n_generated = 0
    queue_len = 0
    path_len = 0
    queue =deque([])
    root=TreeNode(problem,problem.initial_state)
    queue.append(root) # push initial state in queue
    while len(queue)>0:
        if len(queue) > queue_len:
            queue_len = len(queue) # finds max queue length

        next=queue.popleft()
        if check_cyclic_repeats(next):
            continue

        n_expanded += 1 # popping indicates expansion if it does not induce a cycle
        if next.goalp(): # does the current state satisfy the goal condition? (defined by TreeNode)
            del(queue) 
            path_len = len(next.path())
            return {
                'path' : next.path(),
                'Nodes Expanded' : n_expanded,
                'Nodes Generated' : n_generated,
                'Max Queue Length' : queue_len,
                'Path Length' : path_len
            }
        else:
            new_nodes=next.generate_new_tree_nodes() 
            n_generated += len(new_nodes) # generate_new_tree_nodes -> lst[], 
            for new_node in new_nodes:
                queue.append(new_node)         
    print('No solution')
    return NULL

In [19]:
def breadth_first_search_repeats(problem):
    n_expanded = 0
    n_generated = 0
    queue_len = 0
    path_len = 0
    queue =deque([])
    seen = set()
    root=TreeNode(problem,problem.initial_state)
    queue.append(root) # push initial state in queue
    while len(queue)>0:
        if len(queue) > queue_len:
            queue_len = len(queue) # finds max queue length

        next=queue.popleft()
        if next.state in seen:
            continue
        
        seen.add(next.state)
        n_expanded += 1 # popping indicates expansion if it does not induce a cycle
        if next.goalp(): # does the current state satisfy the goal condition? (defined by TreeNode)
            del(queue) 
            path_len = len(next.path())
            return {
                'path' : next.path(),
                'Nodes Expanded' : n_expanded,
                'Nodes Generated' : n_generated,
                'Max Queue Length' : queue_len,
                'Path Length' : path_len
            }
        else:
            new_nodes=next.generate_new_tree_nodes() 
            n_generated += len(new_nodes) # generate_new_tree_nodes -> lst[], 
            for new_node in new_nodes:
                queue.append(new_node)         
    print('No solution')
    return NULL

In [73]:

def depth_first_search_limit(problem, limit):
    n_expanded = 0
    n_generated = 0
    queue_len = 0
    path_len = 0
    found = 0
    queue =deque([])
    root=TreeNode(problem,problem.initial_state)
    queue.append(root) # push initial state in queue
    best_pathes = HashTable()

    while len(queue)>0:
        if len(queue) > queue_len:
            queue_len = len(queue) # finds max queue length

        next=queue.pop()
        if next.g > limit:
            continue
        
        n_expanded += 1 # popping indicates expansion
        path_len = len(next.path())

        if best_pathes.in_hashp(next.state): # if node in ht
            if best_pathes.get_hash_value(next.state) > path_len: # if shorter path, update path len
                best_pathes.add_hash(item = next.state, value = len(next.path()))
            else:
                continue # otherwise skip this node
        else:  # add if not in table
            best_pathes.add_hash(item = next.state, value = len(next.path()))

        if next.goalp(): 
            found = True # have we found a single goal condition?
            found = next # assign to the node; if a worse path to the goal is found again it will not reach this point

        # continue searching bc DFS may not return best sol
        new_nodes=next.generate_new_tree_nodes() 
        n_generated += len(new_nodes) # generate_new_tree_nodes -> lst[], 
        for new_node in new_nodes:
            queue.append(new_node)  
                       
    if found: 
        return {
            'path' : found.path(),
            'Nodes Expanded' : n_expanded,
            'Nodes Generated' : n_generated,
            'Max Queue Length' : queue_len,
            'Path Length' : len(found.path())
        }
    else: 
        print('No solution')
    return NULL

In [74]:
problem=Puzzle8_Problem(Example1) 
output=  depth_first_search_limit(problem, 10)
print('Solution Example 1:')

for k,v in output.items():
    if k == 'path': continue
    
    print(f"{k}: {v}")

print_path(output['path'])

Solution Example 1:
Nodes Expanded: 1379
Nodes Generated: 2332
Max Queue Length: 17
Path Length: 3
(1, 2, 3)
(4, 6, 0)
(7, 5, 8)
 
(1, 2, 3)
(4, 0, 6)
(7, 5, 8)
 
(1, 2, 3)
(4, 5, 6)
(7, 0, 8)
 
(1, 2, 3)
(4, 5, 6)
(7, 8, 0)
 


In [60]:
problem=Puzzle8_Problem(Example3) 
output=  breadth_first_search_stats(problem)
print('Solution Example 3:')

for k,v in output.items():
    if k == 'path': continue
    
    print(f"{k}: {v}")

print_path(output['path'])

Solution Example 3:
Nodes Expanded: 1963
Nodes Generated: 5622
Max Queue Length: 3661
Path Length: 9
(4, 1, 2)
(7, 6, 3)
(0, 5, 8)
 
(4, 1, 2)
(0, 6, 3)
(7, 5, 8)
 
(0, 1, 2)
(4, 6, 3)
(7, 5, 8)
 
(1, 0, 2)
(4, 6, 3)
(7, 5, 8)
 
(1, 2, 0)
(4, 6, 3)
(7, 5, 8)
 
(1, 2, 3)
(4, 6, 0)
(7, 5, 8)
 
(1, 2, 3)
(4, 0, 6)
(7, 5, 8)
 
(1, 2, 3)
(4, 5, 6)
(7, 0, 8)
 
(1, 2, 3)
(4, 5, 6)
(7, 8, 0)
 


In [61]:
problem=Puzzle8_Problem(Example3) 
output=  breadth_first_search_cycles(problem)
print('Solution Example 3:')

for k,v in output.items():
    if k == 'path': continue
    
    print(f"{k}: {v}")

print_path(output['path'])

Solution Example 3:
Nodes Expanded: 177
Nodes Generated: 500
Max Queue Length: 226
Path Length: 9
(4, 1, 2)
(7, 6, 3)
(0, 5, 8)
 
(4, 1, 2)
(0, 6, 3)
(7, 5, 8)
 
(0, 1, 2)
(4, 6, 3)
(7, 5, 8)
 
(1, 0, 2)
(4, 6, 3)
(7, 5, 8)
 
(1, 2, 0)
(4, 6, 3)
(7, 5, 8)
 
(1, 2, 3)
(4, 6, 0)
(7, 5, 8)
 
(1, 2, 3)
(4, 0, 6)
(7, 5, 8)
 
(1, 2, 3)
(4, 5, 6)
(7, 0, 8)
 
(1, 2, 3)
(4, 5, 6)
(7, 8, 0)
 


In [62]:
problem=Puzzle8_Problem(Example3) 
output=  breadth_first_search_repeats(problem)
print('Solution Example 2:')

for k,v in output.items():
    if k == 'path': continue
    
    print(f"{k}: {v}")

print_path(output['path'])

Solution Example 2:
Nodes Expanded: 169
Nodes Generated: 474
Max Queue Length: 204
Path Length: 9
(4, 1, 2)
(7, 6, 3)
(0, 5, 8)
 
(4, 1, 2)
(0, 6, 3)
(7, 5, 8)
 
(0, 1, 2)
(4, 6, 3)
(7, 5, 8)
 
(1, 0, 2)
(4, 6, 3)
(7, 5, 8)
 
(1, 2, 0)
(4, 6, 3)
(7, 5, 8)
 
(1, 2, 3)
(4, 6, 0)
(7, 5, 8)
 
(1, 2, 3)
(4, 0, 6)
(7, 5, 8)
 
(1, 2, 3)
(4, 5, 6)
(7, 0, 8)
 
(1, 2, 3)
(4, 5, 6)
(7, 8, 0)
 


In [46]:
problem=Puzzle8_Problem(Example4) 
output=  breadth_first_search(problem)
print('Solution Example 4:')
print_path(output)

Solution Example 4:
(4, 1, 2)
(7, 6, 3)
(5, 8, 0)
 
(4, 1, 2)
(7, 6, 3)
(5, 0, 8)
 
(4, 1, 2)
(7, 6, 3)
(0, 5, 8)
 
(4, 1, 2)
(0, 6, 3)
(7, 5, 8)
 
(0, 1, 2)
(4, 6, 3)
(7, 5, 8)
 
(1, 0, 2)
(4, 6, 3)
(7, 5, 8)
 
(1, 2, 0)
(4, 6, 3)
(7, 5, 8)
 
(1, 2, 3)
(4, 6, 0)
(7, 5, 8)
 
(1, 2, 3)
(4, 0, 6)
(7, 5, 8)
 
(1, 2, 3)
(4, 5, 6)
(7, 0, 8)
 
(1, 2, 3)
(4, 5, 6)
(7, 8, 0)
 


In [47]:
# problem=Puzzle8_Problem(Example5) 
# output=  breadth_first_search(problem)
# print('Solution Example 5:')
# print_path(output)